# Computing overall return period

In [1]:
%load_ext jupyter_black

import pandas as pd
import numpy as np

In [2]:
# Lists of storm events
readiness_storms = ["FAVIO", "IRINA", "DINEO", "IDAI", "ELOISE"]
action_storms = [
    "FAVIO",
    "JOKWE",
    "DINEO",
    "IDAI",
    "ELOISE",
    "GOMBE",
    "FREDDY",
]
obs_ws_storms = ["DINEO", "IDAI", "ELOISE", "FREDDY", "FILIPO"]
obs_rain_storms = ["FAVIO", "ANA", "GOMBE", "FREDDY"]

# Complete list of storms
all_storms = [
    "FAVIO",
    "JOKWE",
    "IZILDA",
    "DANDO",
    "IRINA",
    "HARUNA",
    "DELIWE",
    "GUITO",
    "HELLEN",
    "CHEDZA",
    "DINEO",
    "DESMOND",
    "IDAI",
    "KENNETH",
    "CHALANE",
    "ELOISE",
    "GUAMBE",
    "ANA",
    "GOMBE",
    "JASMINE",
    "FREDDY",
    "FILIPO",
]

# Create DataFrame
df = pd.DataFrame(
    {
        "storm": all_storms,
        "Readiness": [storm in readiness_storms for storm in all_storms],
        "Action": [storm in action_storms for storm in all_storms],
        "Observational Rainfall": [
            storm in obs_rain_storms for storm in all_storms
        ],
        "Observational Wind Speed": [
            storm in obs_ws_storms for storm in all_storms
        ],
        "Total Affected": [
            162770,
            220013,
            7103,
            40042,
            4958,
            None,
            None,
            None,
            None,
            None,
            750102,
            None,
            1628167,
            400094,
            73254,
            481901,
            None,
            185429,
            736015,
            None,
            1143569,
            50781,
        ],
        "CERF Allocations": [
            1070014,
            548913,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            None,
            2000095,
            None,
            14018121,
            9964907,
            None,
            None,
            None,
            None,
            4018682,
            None,
            9995213,
            None,
        ],
    }
)

In [7]:
# Sort the DataFrame by 'Total Affected' in descending order
# Round values in 'Total Affected' and 'CERF Allocations' columns
df_sorted = df.sort_values(by="Total Affected", ascending=False)


# Define functions for highlighting and coloring bars
def highlight_true(val):
    color = "red" if val else ""
    return f"background-color: {color}"


def color_bar_affected(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, orange {val/df_sorted["Total Affected"].max()*100}%, transparent {val/df_sorted["Total Affected"].max()*100}%);'
    return ""


def color_bar_cerf(val):
    if isinstance(val, (int, float)) and not pd.isna(val):
        return f'background: linear-gradient(90deg, green {val/df_sorted["CERF Allocations"].max()*100}%, transparent {val/df_sorted["CERF Allocations"].max()*100}%);'
    return ""


# Apply styling
styled_df = (
    df_sorted.style.map(
        highlight_true,
        subset=[
            "Readiness",
            "Action",
            "Observational Rainfall",
            "Observational Wind Speed",
        ],
    )
    .map(color_bar_affected, subset=["Total Affected"])
    .map(color_bar_cerf, subset=["CERF Allocations"])
    .format(
        {
            "Total Affected": lambda x: (
                f"{int(x):,}" if pd.notna(x) else ""
            ),  # Format with commas, no decimals, NaN as blank
            "CERF Allocations": lambda x: (
                f"{int(x):,}" if pd.notna(x) else ""
            ),  # Format with commas, no decimals, NaN as blank
        }
    )
    .set_table_styles(
        {"": [{"selector": "table", "props": "background-color: white;"}]}
    )
)

# Display the styled DataFrame
styled_df

,storm,Readiness,Action,Observational Rainfall,Observational Wind Speed,Total Affected,CERF Allocations
12,IDAI,True,True,False,True,"1,628,167","14,018,121"
20,FREDDY,False,True,True,True,"1,143,569","9,995,213"
10,DINEO,True,True,False,True,"750,102","2,000,095"
18,GOMBE,False,True,True,False,"736,015","4,018,682"
15,ELOISE,True,True,False,True,"481,901",
13,KENNETH,False,False,False,False,"400,094","9,964,907"
1,JOKWE,False,True,False,False,"220,013","548,913"
17,ANA,False,False,True,False,"185,429",
0,FAVIO,True,True,True,False,"162,770","1,070,014"
14,CHALANE,False,False,False,False,"73,254",


Return Period for Readiness

In [69]:
num_years_storms = 18
num_ecmwf_years = 45
readiness_rate = (df[df["Readiness"]].shape[0]) / num_years_storms
print(f"The readiness activation rate is {round(readiness_rate*100,1)}%")
print(f"The readiness return period is 1-in-{round(1/readiness_rate, 1)} years")

The readiness activation rate is 27.8%
The readiness return period is 1-in-3.6 years


Return Period for Action: 4 years

In [70]:
action_rate = 1 / 4
print(f"The action activation rate is {round(action_rate*100,1)}%")
print(f"The action return period is 1-in-{round(1/action_rate, 1)} years")

The action activation rate is 25.0%
The action return period is 1-in-4.0 years


Return Period for Readiness + Action

In [71]:
readiness_and_action = (
    df[(df["Readiness"]) & (df["Action"])].shape[0]
) / num_years_storms
readiness_action_rate = readiness_rate + action_rate - readiness_and_action
print(
    f"The readiness or action activation rate is {round(readiness_action_rate*100,1)}%"
)
print(
    f"The readiness or action return period is 1-in-{round(1/readiness_action_rate, 1)} years"
)

The readiness or action activation rate is 30.6%
The readiness or action return period is 1-in-3.3 years


Return Period for Observational : Wind Speed >= 48 knots

In [72]:
obs_ws_rate = (df[df["Observational Wind Speed"]].shape[0]) / num_years_storms
print(f"The observational wind speed activation rate is {round(obs_ws_rate*100,1)}%")
print(
    f"The observational wind speed return period is 1-in-{round(1/obs_ws_rate, 1)} years"
)

The observational wind speed activation rate is 27.8%
The observational wind speed return period is 1-in-3.6 years


Return Period for Observational : Rainfall >= 40 mm

In [73]:
obs_rain_rate = (df[df["Observational Rainfall"]].shape[0]) / num_years_storms
print(f"The observational rainfall activation rate is {round(obs_rain_rate*100,1)}%")
print(
    f"The observational rainfall return period is 1-in-{round(1/obs_rain_rate, 1)} years"
)

The observational rainfall activation rate is 22.2%
The observational rainfall return period is 1-in-4.5 years


Return Period for Observational

In [74]:
observational = (
    df[(df["Observational Rainfall"]) | (df["Observational Wind Speed"])].shape[0]
) / num_years_storms
print(f"The observational rainfall activation rate is {round(observational*100,1)}%")
print(
    f"The observational rainfall return period is 1-in-{round(1/observational, 1)} years"
)

The observational rainfall activation rate is 44.4%
The observational rainfall return period is 1-in-2.2 years


Return Period for Readiness + Action + Observational

In [85]:
readiness_and_action_obs = (
    df[
        ((df["Readiness"]) | (df["Action"]))
        & ((df["Observational Rainfall"]) | (df["Observational Wind Speed"]))
    ].shape[0]
) / num_years_storms
readiness_action_rate = readiness_and_action + observational - readiness_and_action_obs
print(
    f"The readiness or action or observational activation rate is {round(readiness_action_rate*100,1)}%"
)
print(
    f"The readiness or action or observational return period is 1-in-{round(1/readiness_action_rate, 1)} years"
)

The readiness or action or observational activation rate is 33.3%
The readiness or action or observational return period is 1-in-3.0 years
